# SEQ08 redux
## testing the fall 2018 sequence processing method on old samples
Because we are now using a targeted set of ~1020 contigs, we can trim, map and call SNPs on individual sequencing runs and delete the intermediate files when done.

## Prepare the directories for sequence storage

In [1]:
!ls /local/shared/pinsky_lab/sequencing/hiseq_2015_05_11_SEQ08/

clownfish-ddradseq-seq08-for-231-cycles-h3mgvbcxx_1_read_1_passed_filter.fastq.gz
clownfish-ddradseq-seq08-for-231-cycles-h3mgvbcxx_1_read_2_index_read_passed_filter.fastq.gz
clownfish-ddradseq-seq08-for-231-cycles-h3mgvbcxx_1_read_3_index_read_passed_filter.fastq.gz
clownfish-ddradseq-seq08-for-231-cycles-h3mgvbcxx_2_read_1_passed_filter.fastq.gz
clownfish-ddradseq-seq08-for-231-cycles-h3mgvbcxx_2_read_2_index_read_passed_filter.fastq.gz
clownfish-ddradseq-seq08-for-231-cycles-h3mgvbcxx_2_read_3_index_read_passed_filter.fastq
namelist



## Go to the Princeton webpage to retrieve the seq data - took about 5 minutes for SEQ18 - 576 samples
https://htseq.princeton.edu/cgi-bin/login.pl?redirect_url=https://htseq.princeton.edu/cgi-bin/dashboard.pl **done**
* Click on the sequencing run of interest in the box on the left that says “Recently Entered Samples"
* In the box titled Sample Provenance, click on the link following "This library was utilized within the following output(s):” - repeat for each lane
* In the “Data and Statistics” box, in the bottom right corner is a green button that says “Download"
* click wget and then the copy button to get all files.  This will copy the wget commands for all files.
* paste in amphiprion **in the directory you made in the previous step**

## Make directories in personal workspace - 4 pools 

In [2]:
%%bash
cd /data/apcl
# mkdir 04seq
cd 08seq/ 
# mkdir logs samples scripts
mkdir bcsplit 01Pool 02Pool 03Pool 04Pool 
cd bcsplit/
mkdir lane1 lane2

mkdir: cannot create directory `bcsplit': File exists
mkdir: cannot create directory `01Pool': File exists
mkdir: cannot create directory `02Pool': File exists
mkdir: cannot create directory `03Pool': File exists
mkdir: cannot create directory `04Pool': File exists
mkdir: cannot create directory `lane1': File exists
mkdir: cannot create directory `lane2': File exists


In [3]:
!ls /data/apcl/08seq/

01Pool	02Pool	03Pool	04Pool	bcsplit  samples


** If the logs and scripts directories aren't there, move them now **

In [4]:
!cp -r ~/02-apcl-ddocent/08seq/logs /data/apcl/08seq/
!cp -r ~/02-apcl-ddocent/08seq/scripts /data/apcl/08seq/
!ls /data/apcl/08seq/logs

08seq-blacklist.txt  31process.out.tsv	barcodes     names_33	    sstacks.out
08seq-loci.txt	     32process.out	index-seq08  names-seq08    stacks
30process.out	     32process.out.tsv	names_30     rxsstacks.out  unpaired
30process.out.tsv    33process.out	names_31     rxstacks	    ustacks.out
31process.out	     33process.out.tsv	names_32     rxstacks.out


In [5]:
!rm /data/apcl/08seq/logs/08seq-blacklist.txt /data/apcl/08seq/logs/08seq-loci.txt /data/apcl/08seq/logs/rxsstacks.out /data/apcl/08seq/logs/rxstacks /data/apcl/08seq/logs/rxstacks.out 

rm: cannot remove `/data/apcl/08seq/logs/rxstacks': Is a directory


## Make sure index file is present
* see seq_proc.Rmd notebook in RStudio to generate index and names files for large batches of pools/samples

## Make sure barcodes file is present

## Run barcode splitter in lane1 folder and lane 2 folder - takes about 8 hours for 192 samples
* Closing the notebook closes this.  
* Paste into terminal window.
* started 11:30am on Monday

In [ ]:
cd /data/apcl/08seq/bcsplit/lane1
nohup /local/home/michelles/14_programs/paired_sequence_utils/barcode_splitter.py \
--bcfile ../../logs/index-seq08 --idxread 2 --suffix .fastq.gz \
/local/shared/pinsky_lab/sequencing/hiseq_2015_05_11_SEQ08/clownfish-ddradseq-seq08-for-231-cycles-h3mgvbcxx_1_read_1_passed_filter.fastq.gz \
/local/shared/pinsky_lab/sequencing/hiseq_2015_05_11_SEQ08/clownfish-ddradseq-seq08-for-231-cycles-h3mgvbcxx_1_read_2_index_read_passed_filter.fastq.gz &

cd /data/apcl/08seq/bcsplit/lane2
nohup /local/home/michelles/14_programs/paired_sequence_utils/barcode_splitter.py \
--bcfile ../../logs/index-seq08 --idxread 2 --suffix .fastq.gz \
/local/shared/pinsky_lab/sequencing/hiseq_2015_05_11_SEQ08/clownfish-ddradseq-seq08-for-231-cycles-h3mgvbcxx_2_read_1_passed_filter.fastq.gz \
/local/shared/pinsky_lab/sequencing/hiseq_2015_05_11_SEQ08/clownfish-ddradseq-seq08-for-231-cycles-h3mgvbcxx_2_read_2_index_read_passed_filter.fastq.gz &

## Check the output statistics in the nohup and move to logs with a rename

In [7]:
!head /data/apcl/08seq/bcsplit/lane1/nohup.out

Traceback (most recent call last):
  File "/local/home/michelles/14_programs/paired_sequence_utils/barcode_splitter.py", line 342, in <module>
    sys.exit(main())
  File "/local/home/michelles/14_programs/paired_sequence_utils/barcode_splitter.py", line 95, in main
    barcode_dict = read_barcodes(options.bcfile)
  File "/local/home/michelles/14_programs/paired_sequence_utils/barcode_splitter.py", line 203, in read_barcodes
    filehandle = open(filename, 'rb')
IOError: [Errno 2] No such file or directory: '../../logs/index-08seq'


In [ ]:
!mv /data/apcl/07seq/bcsplit/lane1/nohup.out /data/apcl/07seq/logs/lane1.out

In [ ]:
!head /data/apcl/07seq/bcsplit/lane2/nohup.out

In [ ]:
!mv /data/apcl/07seq/bcsplit/lane2/nohup.out /data/apcl/07seq/logs/lane2.out

## Concatenate the results in the bcsplit folder - took about 1 minute

In [ ]:
%%bash
cd /data/apcl/07seq/bcsplit/
cat ./lane1/P016-read-1.fastq.gz ./lane2/P016-read-1.fastq.gz > ../01Pool/P016.fastq.gz
cat ./lane1/P017-read-1.fastq.gz ./lane2/P017-read-1.fastq.gz > ../02Pool/P017.fastq.gz
cat ./lane1/P018-read-1.fastq.gz ./lane2/P018-read-1.fastq.gz > ../03Pool/P018.fastq.gz
cat ./lane1/P019-read-1.fastq.gz ./lane2/P019-read-1.fastq.gz > ../04Pool/P019.fastq.gz

## Run process radtags with script - takes ~2 hours for 192 samples
* Had to **nano the script** to point to the correct place and changed my email to gmail so I will get it on my phone (if it sends)
* If this notebook closes, these processes stop - **ended up pasting this into a terminal window** because I was afraid the network would drop and the process would stop

* Started at 6:42am on Tuesday
* finished between 7:45am and 9:15am on Tuesday

In [ ]:
!ls /data/apcl/07seq/scripts

%%bash
cd /data/apcl/07seq/01Pool/
nohup ../scripts/16process.sh &
cd /data/apcl/07seq/02Pool/
nohup ../scripts/17process.sh &
cd /data/apcl/07seq/03Pool/
nohup ../scripts/18process.sh &
cd /data/apcl/07seq/04Pool/
nohup ../scripts/19process.sh &

## Move the process_radtags logs to the logs folder - wrote command lines for large batches in seq_proc.Rmd notebook and saved the lines in files called SEQXX-move_radlogs.sh 

The script did this for us so we don't have to

In [ ]:
#%%bash
#mv 01Pool/process_radtags.log ./logs/processP012.log
#mv 02Pool/process_radtags.log ./logs/processP013.log
#mv 03Pool/process_radtags.log ./logs/processP014.log
#mv 04Pool/process_radtags.log ./logs/processP015.log

In [ ]:
!ls /data/apcl/07seq/logs

## Run the readprocesslog.py script to convert the process log into a tsv that can be imported into the database through R
* This was done years ago, not doing now.

%%bash
cd ~/02-apcl-ddocent/04seq/01Pool/
readprocesslog.py ../logs/20process.out
cd ~/02-apcl-ddocent/04seq/02Pool/
readprocesslog.py ../logs/21process.out
cd ~/02-apcl-ddocent/04seq/03Pool/
readprocesslog.py ../logs/22process.out
cd ~/02-apcl-ddocent/04seq/04Pool/
readprocesslog.py ../logs/23process.out

## Move the newly created tsvs to the laptop with fetch 
* import them into the database using the R notebook script read_to_db.Rmd

## Rename the sample files
* **recreate the names files using only the ligation id**
* current names files have sample id included
* using the seq_proc.Rmd script in the methods repository > procedural notebooks

In [ ]:
%%bash
cd /data/apcl/07seq/01Pool/
sh rename.for.dDocent_se_gz ../logs/names_016.tsv
mv APCL* /data/apcl/07seq/samples/
cd /data/apcl/07seq/02Pool/
sh rename.for.dDocent_se_gz ../logs/names_017.tsv
mv APCL* /data/apcl/07seq/samples/
cd /data/apcl/07seq/03Pool/
sh rename.for.dDocent_se_gz ../logs/names_018.tsv
mv APCL* /data/apcl/07seq/samples/
cd /data/apcl/07seq/04Pool/
sh rename.for.dDocent_se_gz ../logs/names_019.tsv
mv APCL* /data/apcl/07seq/samples/

## Once all of the samples have been moved, delete the pool directories

In [ ]:
!rm -r /data/apcl/07seq/*Pool

## Identify regenotyped samples, cat and move them.

* using the R script cat_regenos.Rmd 
* files were all cat'd during seq28
* still need to be moved

In [ ]:
!mkdir /data/apcl/07seq/samples/regenos

In [ ]:
%%bash
#!/bin/bash
mv /data/apcl/07seq/samples/APCL_L0448* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0449* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0455* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0485* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0486* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1025* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1026* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1027* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1028* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1029* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1030* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1031* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1032* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1033* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1034* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1035* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1036* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1037* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1038* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1039* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1040* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1041* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1042* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1043* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1044* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1045* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1046* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1047* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1049* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1050* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1051* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1052* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1053* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1054* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1055* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1056* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1057* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1058* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1059* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1060* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1061* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1062* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1063* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1064* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1065* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1066* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1067* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1068* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1069* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1070* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1086* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1098* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1106* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1023* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1024* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L1048* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0450* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0452* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0453* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0466* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0597* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0606* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0610* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0637* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0638* /data/apcl/07seq/samples/regenos/
mv /data/apcl/07seq/samples/APCL_L0595* /data/apcl/07seq/samples/regenos/


## Wait for 2 more seqs so there are 3 runs before trim, map and call